In [1]:
import json
from openai import OpenAI
import pandas as pd
import numpy as np

### 1. 提取DuReaderQG数据集中的语境和问题

In [ ]:
# 读取原始的train.json文件
with open('./datasets/DuReaderQG/train.json', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f.readlines()[:1500]]

# 抽取前500条记录的context内容
gen_context_data = []
for i, record in enumerate(data[:500]):
    gen_context_data.append({
        "context": record["context"],
        "id": i + 1  # 重新索引id，从1开始
    })

# 将抽取的内容保存到gen_context.json文件中
with open('gen_context.json', 'w', encoding='utf-8') as f:
    json.dump(gen_context_data, f, ensure_ascii=False, indent=4)

print("gen_context.json has been generated.")

# 抽取第1001到1500条记录的question内容
short_questions_data = []
for i, record in enumerate(data[1000:1500]):  # 注意Python切片是左闭右开的，所以这里使用1000:1500来获取1001-1500条记录
    short_questions_data.append({
        "question": record["question"],
        "id": i + 1001  # 重新索引id，从1001开始
    })

# 将抽取的内容保存到short_questions.json文件中
with open('short_questions.json', 'w', encoding='utf-8') as f:
    json.dump(short_questions_data, f, ensure_ascii=False, indent=4)

print("short_questions.json has been generated.")

### 2. 利用语境生成500条问题

In [ ]:
ds_key = ""
ds_url = "https://api.deepseek.com/"
ds_mod = "deepseek-chat"
local_url = "http://localhost:1234/v1"
local_key = "lm-studio"
local_mod = "Qwen/Qwen1.5-14B-Chat-GGUF"
# 读取json文件
with open('./gen_context.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 创建一个空列表来保存模型的响应
responses = []

# 遍历每个context
for item in data:
    context = item['context']
    id = item['id']
    
    client = OpenAI(   
                api_key=local_key, 
                base_url=local_url
            )
    
    # 发送请求到模型
    response = client.chat.completions.create(
        model=local_mod,
        messages=[
            {
                "role": "system", 
                "content": "根据用户的输入文本，提出一个有价值的问题，务必使用**中文**表述问题，仅直接简要表达问题即可，不许给出答案"
            },
            {
                "role": "user", 
                "content": context},
        ],
        max_tokens=512,
        frequency_penalty=1.2,
    )
    
    # 保存模型的响应
    responses.append({
        "id": id,
        "question": response.choices[0].message.content
    })

# 将响应保存到long_question.json文件中
with open('long_questions.json', 'w', encoding='utf-8') as file:
    json.dump(responses, file, ensure_ascii=False, indent=4)

### 3. 合并DuReaderQG的现成问题与生成问题作为反例

In [2]:
# 读取两个JSON文件
with open('long_questions.json', 'r', encoding='utf-8') as f:
    long_questions = json.load(f)

with open('short_questions.json', 'r',  encoding='utf-8') as f:
    short_questions = json.load(f)

# 合并两个列表
questions = long_questions + short_questions

# 创建DataFrame
false_df = pd.DataFrame(questions)

# 提取"question"字段并创建"text"列
false_df['text'] = false_df['question']

# 创建"label"列并全部设置为0
false_df['label'] = 0

# 删除原始的"question"列（如果不需要保留）
false_df.drop(columns=['question'], inplace=True)
false_df.drop(columns=['id'], inplace=True)
# 显示DataFrame
false_df.head()

,text,label
0,雪见为何醒来后一脸木然?,0
1,方太燃气热水器为什么能解决忽热忽冷的问题？,0
2,乔丹在第一次退役之前拿过几次常规赛MVP？,0
3,杜达昌在重庆曙光男科医院做哪个科室的？,0
4,吴磊参演的《斗破苍穹》是什么时候定档？,0


### 4. 加入竞赛中的示例正例形成完整数据集

In [3]:
# 比赛用数据
with open('question.json', 'r',  encoding='utf-8') as f:
    true_questions = [json.loads(line) for line in f.readlines()]
# 创建DataFrame
true_df = pd.DataFrame(true_questions)
# 提取"question"字段并创建"text"列
true_df['text'] = true_df['question']
# 创建"label"列并全部设置为1
true_df['label'] = 1
# 删除原始的"question"列（如果不需要保留）
true_df.drop(columns=['question'], inplace=True)
true_df.drop(columns=['id'], inplace=True)

# print(true_df)

# 合并true_df和false_df
merged_df = pd.concat([true_df, false_df]).reset_index(drop=True)

# 显示合并后的DataFrame
# merged_df.head()
# merged_df.tail()

### 5. 划分训练集和测试集并优化格式

In [5]:
# 设置随机种子以确保结果可重复
np.random.seed(42)

# 计算划分点
total_data = len(merged_df)
split_point_test = int(total_data * 1 / 10)  # 1/10 的数据将用于测试集
split_point_val = int(total_data * 1 / 10)  # 1/10 的数据将用于验证集

# 随机打乱数据
shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)

# 划分训练集、验证集和测试集
test_set = shuffled_df.iloc[:split_point_test]
val_set = shuffled_df.iloc[split_point_test:split_point_test + split_point_val]
train_set = shuffled_df.iloc[split_point_test + split_point_val:]

# 显示划分后的数据集
print("测试集:")
print(test_set.head())

print("验证集:")
print(val_set.head())

print("训练集:")
print(train_set.head())

# 转换为parquet格式
test_set.to_parquet('./datasets/FinQuestions_v2/test.parquet')
val_set.to_parquet('./datasets/FinQuestions_v2/val.parquet')
train_set.to_parquet('./datasets/FinQuestions_v2/train.parquet')

测试集:
                                                text  label
0                                         上海打乙肝疫苗多少钱      0
1                          云南沃森生物技术股份有限公司创始人的研发资质如何？      1
2                             怎样在西北街证券开户时输入东吴证券的客户号？      0
3  请问2020年二季度有多少家基金是净申购?它们的净申购份额加起来是多少?请四舍五入保留小数点两位。      1
4                                   如何更改魔兽争霸3的视频分辨率？      0
验证集:
                                                  text  label
200                                           寻龙诀 蒙古插曲      0
201                    新疆浩源天然气股份有限公司实际控制人直系亲属控股的企业有哪些？      1
202             请帮我查询在2020年,信达澳亚基金管理有限公司成立哪种类型的基金个数最多?      1
203                  西安启源机电装备股份有限公司上游行业发展状况对本行业制约情况如何？      1
204  请帮我查询下，在20211117，申万行业分类里一级行业为商业贸易行业的所有股票里, 成交金...      1
训练集:
                                                  text  label
400                       20191231日，创世纪在多少只基金的前十大重仓股中？      1
401                        湖南南岭民用爆破器材股份有限公司拥有的核心技术有哪些？      1
402                          招商基金管理有限公司管理的混合型产品的数量有多少?      1
403  